In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from six.moves import cPickle as pickle
import os

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_df = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
test_df = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])
macro_df = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])
train_df = pd.merge(train_df, macro_df, how='left', on='timestamp')
test_df = pd.merge(test_df, macro_df, how='left', on='timestamp')
print(train_df.shape, test_df.shape)

(30471, 391) (7662, 390)


In [3]:

# truncate the extreme values in price_doc #
ulimit = np.percentile(train_df.price_doc.values, 99)
llimit = np.percentile(train_df.price_doc.values, 1)
train_df.loc[train_df['price_doc']>ulimit,'price_doc'] = ulimit
train_df.loc[train_df['price_doc']<llimit,'price_doc'] = llimit

In [4]:
#remove the special characters first
try:
    train_df.loc[train_df['child_on_acc_pre_school']== '#!','child_on_acc_pre_school'] = '0'
    test_df.loc[test_df['child_on_acc_pre_school']== '#!','child_on_acc_pre_school'] = '0'
except:
    pass

In [5]:
def rep_commas(a):
    return a.replace(',','')

train_df.child_on_acc_pre_school= train_df.child_on_acc_pre_school.fillna(0).astype(str).apply(rep_commas).astype(int)
train_df.modern_education_share = train_df.modern_education_share.fillna(0).astype(str).apply(rep_commas).astype(int)
train_df.old_education_build_share  =train_df.old_education_build_share.fillna(0).astype(str).apply(rep_commas).astype(int)
test_df.child_on_acc_pre_school= test_df.child_on_acc_pre_school.fillna(0).astype(str).apply(rep_commas).astype(int)
test_df.modern_education_share = test_df.modern_education_share.fillna(0).astype(str).apply(rep_commas).astype(int)
test_df.old_education_build_share  =test_df.old_education_build_share.fillna(0).astype(str).apply(rep_commas).astype(int)

In [6]:

for f in train_df.columns:
    if train_df[f].dtype=='object':
        print(f)
        lbl = LabelEncoder()
        lbl.fit(list(train_df[f].values.astype('str')) + list(test_df[f].values.astype('str')))
        train_df[f] = lbl.transform(list(train_df[f].values.astype('str')))
        test_df[f] = lbl.transform(list(test_df[f].values.astype('str')))

product_type
sub_area
culture_objects_top_25
thermal_power_plant_raion
incineration_raion
oil_chemistry_raion
radiation_raion
railroad_terminal_raion
big_market_raion
nuclear_reactor_raion
detention_facility_raion
water_1line
big_road1_1line
railroad_1line
ecology


In [12]:
def add_count(df, group_col):
    grouped_df = df.groupby(group_col)["id"].aggregate("count").reset_index()
    grouped_df.columns = [group_col, "count_"+group_col]
    df = pd.merge(df, grouped_df, on=group_col, how="left")
    return df

In [13]:
def make_features(df):
    df["null_count"] = df.isnull().sum(axis=1)
    df.fillna(-99, inplace=True)
    
    df.loc[df["build_year"] == 0.0,"build_year"] = 2000.0
    df.loc[df["build_year"] == 20052009.0 ,"build_year"] = 2005.0
    df.loc[df["build_year"] == 215.0 ,"build_year"] = 2015.0
    df.loc[df["build_year"] == 4965.0 ,"build_year"] = 1965.0
    df.loc[df["build_year"] == 71.0 ,"build_year"] = 1971.0
    df.loc[df["build_year"] == 1.0 ,"build_year"] = 2001.0
    df.loc[df["build_year"] == 3.0 ,"build_year"] = 2003.0
    df.loc[df["build_year"] == 20.0 ,"build_year"] = 2000.0
    df.loc[df["material"] == 3.0,"material"] = 1.0
    df.loc[((df["material"] == 5.0) & (df["build_year"] == -99.0)),"build_year"] = 1970.0
    df.loc[((df["material"] == 2.0 )& (df["build_year"] == -99.0)),"build_year"] = 1965.0
    df.loc[((df["material"] == 4.0) & (df["build_year"] == -99.0)),"build_year"] = 2016.0
    df.loc[((df["material"] == 6.0) & (df["build_year"] == -99.0)),"build_year"] = 2016.0
    df.loc[:,'male_f_p' ]= df.loc[:,'male_f']/np.maximum(df.loc[:,'full_all'],1)

    df.loc[:,'young_all_p' ]= df.loc[:,'young_all']/np.maximum(df.loc[:,'full_all'],1)
    df.loc[:,'young_male_p' ]= df.loc[:,'young_male']/np.maximum(df.loc[:,'full_all'],1)

    df.loc[:,'work_all_p' ]= df.loc[:,'work_all']/np.maximum(df.loc[:,'full_all'],1)
    df.loc[:,'work_male_p' ]= df.loc[:,'work_male']/np.maximum(df.loc[:,'full_all'],1)

    df.loc[:,'ekder_all_p' ]= df.loc[:,'ekder_all']/np.maximum(df.loc[:,'full_all'],1)
    df.loc[:,'ekder_male_p' ]= df.loc[:,'ekder_male']/np.maximum(df.loc[:,'full_all'],1)

    df.loc[:,'0_6_all_p' ]= df.loc[:,'0_6_all']/np.maximum(df.loc[:,'full_all'],1)
    df.loc[:,'0_6_male_p' ]= df.loc[:,'0_6_male']/np.maximum(df.loc[:,'full_all'],1)

    df.loc[:,'7_14_all_p' ]= df.loc[:,'7_14_all']/np.maximum(df.loc[:,'full_all'],1)
    df.loc[:,'7_14_male_p' ]= df.loc[:,'7_14_male']/np.maximum(df.loc[:,'full_all'],1)

    df.loc[:,'0_17_all_p' ]= df.loc[:,'0_17_all']/np.maximum(df.loc[:,'full_all'],1)
    df.loc[:,'0_17_male_p' ]= df.loc[:,'0_17_male']/np.maximum(df.loc[:,'full_all'],1)

    df.loc[:,'16_29_all_p' ]= df.loc[:,'16_29_all']/np.maximum(df.loc[:,'full_all'],1)
    df.loc[:,'16_29_male_p' ]= df.loc[:,'16_29_male']/np.maximum(df.loc[:,'full_all'],1)

    df.loc[:,'0_13_all_p' ]= df.loc[:,'0_13_all']/np.maximum(df.loc[:,'full_all'],1)
    df.loc[:,'0_13_male_p' ]= df.loc[:,'0_13_male']/np.maximum(df.loc[:,'full_all'],1)



    # year and month #
    df["yearmonth"] = df["timestamp"].dt.year*100 + df["timestamp"].dt.month
    # year and week #
    df["yearweek"] = df["timestamp"].dt.year*100 + df["timestamp"].dt.weekofyear
    # year #
    df["year"] = df["timestamp"].dt.year
    # month of year #
    df["month_of_year"] = df["timestamp"].dt.month
    # week of year #
    df["week_of_year"] = df["timestamp"].dt.weekofyear
    # day of week #
    df["day_of_week"] = df["timestamp"].dt.weekday
    # ratio of living area to full area #
    df["ratio_life_sq_full_sq"] = df["life_sq"] / np.maximum(df["full_sq"].astype("float"),1)

    df.loc[df["ratio_life_sq_full_sq"]<0,"ratio_life_sq_full_sq"] = 0
    df.loc[df["ratio_life_sq_full_sq"]>1,"ratio_life_sq_full_sq"] = 1
    # ratio of kitchen area to living area #
    df["ratio_kitch_sq_life_sq"] = df["kitch_sq"] / np.maximum(df["life_sq"].astype("float"),1)

    df.loc[df["ratio_kitch_sq_life_sq"]<0,"ratio_kitch_sq_life_sq"] = 0
    df.loc[df["ratio_kitch_sq_life_sq"]>1,"ratio_kitch_sq_life_sq"] = 1
    # ratio of kitchen area to full area #
    df["ratio_kitch_sq_full_sq"] = df["kitch_sq"] / np.maximum(df["full_sq"].astype("float"),1)

    df.loc[df["ratio_kitch_sq_full_sq"]<0,"ratio_kitch_sq_full_sq"] = 0
    df.loc[df["ratio_kitch_sq_full_sq"]>1,"ratio_kitch_sq_full_sq"] = 1
    # floor of the house to the total number of floors in the house #
    df["ratio_floor_max_floor"] = df["floor"] / df["max_floor"].astype("float")
    
    # num of floor from top #
    df["floor_from_top"] = df["max_floor"] - df["floor"]
    df["extra_sq"] = df["full_sq"] - df["life_sq"]
    df["age_of_building"] = df["build_year"] -df["year"]
    df["ratio_preschool"] = df["children_preschool"] / df["preschool_quota"].astype("float")
    df["ratio_school"] = df["children_school"] / df["school_quota"].astype("float")
    df["ratio_preschool"] = df["children_preschool"] / df["preschool_quota"].astype("float")
    df["ratio_school"] = df["children_school"] / df["school_quota"].astype("float")
    df = add_count(df, "yearmonth")
    df = add_count(df, "yearweek")

    return df

train_df = make_features(train_df)
test_df = make_features(test_df)

In [14]:
train_X = train_df.drop(["id","ID_metro","ID_railroad_station_walk","ID_railroad_station_avto",
                         "ID_big_road1","ID_big_road2","ID_railroad_terminal","ID_bus_terminal",
                         "timestamp",'male_f','female_f','young_all','young_male','young_female',
                         'work_all','work_male','work_female','ekder_all','ekder_male','ekder_female',
                         '0_6_all','0_6_male','0_6_female','7_14_all','7_14_male','7_14_female','0_17_all',
                         '0_17_male','0_17_female','16_29_all','16_29_male','16_29_female','0_13_all',
                         '0_13_male','0_13_female', "price_doc"], axis=1)
test_X = test_df.drop(["id","ID_metro","ID_railroad_station_walk","ID_railroad_station_avto",
                         "ID_big_road1","ID_big_road2","ID_railroad_terminal","ID_bus_terminal",
                         "timestamp",'male_f','female_f','young_all','young_male','young_female',
                         'work_all','work_male','work_female','ekder_all','ekder_male','ekder_female',
                         '0_6_all','0_6_male','0_6_female','7_14_all','7_14_male','7_14_female','0_17_all',
                         '0_17_male','0_17_female','16_29_all','16_29_male','16_29_female','0_13_all',
                         '0_13_male','0_13_female'] , axis=1)

train_y = np.log1p(train_df.price_doc.values)

In [15]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear'}

In [ ]:
optimized_GBM = GridSearchCV(xgb.XGBRegressor(**ind_params), cv_params, scoring = 'neg_mean_squared_error', cv = 5, n_jobs = -1) 
optimized_GBM.fit(train_X, train_y)

In [ ]:
print(optimized_GBM.grid_scores_)